In [ ]:
!pip install datasets
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.2 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset

# Load FEVER dataset with a specific config name ('v1.0' or 'v2.0')
fever_train = load_dataset("fever", "v1.0", split="train")
fever_val = load_dataset("fever", "v1.0", split="labelled_dev")

# Exclude examples with the label 'NOT ENOUGH INFO'
fever_train = fever_train.filter(lambda example: example["label"] != "NOT ENOUGH INFO")
fever_val = fever_val.filter(lambda example: example["label"] != "NOT ENOUGH INFO")

# Define label list and convert labels to integers
label_list = ["SUPPORTS", "REFUTES"]

# Rename column and convert labels to integers
fever_train = fever_train.rename_column("label", "labels")
fever_val = fever_val.rename_column("label", "labels")

# Create a new dataset with the modified labels
fever_train = fever_train.map(lambda example: {"label": label_list.index(example["labels"])})
fever_val = fever_val.map(lambda example: {"label": label_list.index(example["labels"])})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for fever contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/fever
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/311431 [00:00<?, ? examples/s]

Generating labelled_dev split:   0%|          | 0/37566 [00:00<?, ? examples/s]

Generating unlabelled_dev split:   0%|          | 0/19998 [00:00<?, ? examples/s]

Generating unlabelled_test split:   0%|          | 0/19998 [00:00<?, ? examples/s]

...

Generating paper_test split:   0%|          | 0/18567 [00:00<?, ? examples/s]

Filter:   0%|          | 0/311431 [00:00<?, ? examples/s]

Filter:   0%|          | 0/37566 [00:00<?, ? examples/s]

Map:   0%|          | 0/263822 [00:00<?, ? examples/s]

Map:   0%|          | 0/28625 [00:00<?, ? examples/s]

In [ ]:
# Load pre-trained model and tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_list))

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize function for mapping with padding
def tokenize_function(examples):
    return tokenizer(examples["claim"], truncation=True, padding="max_length", max_length=64)

# Tokenize the datasets
fever_train_tokenized = fever_train.map(tokenize_function, batched=True)
fever_val_tokenized = fever_val.map(tokenize_function, batched=True)

Map:   0%|          | 0/263822 [00:00<?, ? examples/s]

Map:   0%|          | 0/28625 [00:00<?, ? examples/s]

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./fever_model",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    save_total_limit=2,
    evaluation_strategy="epoch",
    num_train_epochs=1,
    logging_dir="./logs",
    report_to="tensorboard",
    push_to_hub=True,
)

In [ ]:
# Set up Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=fever_train_tokenized,
    eval_dataset=fever_val_tokenized,
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.170800,0.506981


Checkpoint destination directory ./fever_model/checkpoint-1500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./fever_model/checkpoint-2000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=16489, training_loss=0.21355953999665733, metrics={'train_runtime': 2210.8425, 'train_samples_per_second': 119.331, 'train_steps_per_second': 7.458, 'total_flos': 4368476756027904.0, 'train_loss': 0.21355953999665733, 'epoch': 1.0})

In [ ]:
# Save the model locally
model.save_pretrained("./final_model")

In [ ]:
# Prediction function with credibility score
def predict_credibility(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=64)

    # Move input tensors to CPU explicitly
    inputs = {key: value.to("cuda") for key, value in inputs.items()}

    with torch.no_grad():
        # Move model to CPU for inference
        model.to("cuda")

        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1).squeeze().tolist()

    misinformation_score = probabilities[label_list.index("REFUTES")] * 100
    good_information_score = probabilities[label_list.index("SUPPORTS")] * 100

    return misinformation_score, good_information_score

In [ ]:
# Example usage
demo_input = "Mahesh is"
misinformation_score, good_information_score = predict_credibility(demo_input)

# Output
print("Demo Input:", demo_input)
print(f"Misinformation Score: {misinformation_score:.2f}%")
print(f"Good Information Score: {good_information_score:.2f}%")

Demo Input: JP is a bootaa gorila
Misinformation Score: 79.46%
Good Information Score: 20.54%


In [ ]:
model.push_to_hub

<bound method PushToHubMixin.push_to_hub of DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_features=768, out_features=3072, bias=True)
            (lin2): Linear(in_features=3072, out_features=768, bias=True)
            (activation): GELUActivation()
          )
          (output_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        )
      )
    )
  )
  (pre_classifier): Linear(in_features=768, out_features=768, bias=True)
  (classifier): Linear(in_features=768, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)>